We use DeepSEA as the biological critic. It's input is encoded A, G, C, T and takes in sequences of length 1000.

We set up our generator to also create one-hot encoded strings of A, G, C, T of length 1000.

# Data preprocessing

In [1]:
from Bio import SeqIO
from textwrap import wrap
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
ngpu=1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Split reference genome into 131-kb samples to match the input size of Basenji

We also convert lower-case soft-masked regions of the DNA to uppercase for proper one-hot encoding 

See: https://bioinformatics.stackexchange.com/questions/225/uppercase-vs-lowercase-letters-in-reference-genome

In [3]:
sample_length = 1000

In [4]:
# nucleic_acids = "ACGT"
# samples = []

# if len(samples) == 0:
#   for i, record in enumerate(SeqIO.parse("/content/GCF_000001405.39_GRCh38.p13_genomic.fna", "fasta")):
#     if i <= 5:
#       sequence = str(record.seq)
#       for sample in wrap(sequence, sample_length):
#         sample = sample.upper()
#         if set(sample) == set(nucleic_acids) and len(sample) == sample_length: # Ensure string only contains proper nucleic acids
#           samples.append(sample)

In [5]:
nucleic_acids = "AGCT"

num_training_samples = 100
validation_samples = []
num_validation_samples = 100

def generate_samples(num_samples):
  samples = []
  for i in range(num_samples):
    sample = ""
    for i in range(sample_length):
      base_selection = int(random.random()*4)
      sample += nucleic_acids[base_selection]
    samples.append(sample)
  return samples

raw_training_samples = generate_samples(num_training_samples)

raw_validation_samples = generate_samples(num_validation_samples)

Initialize one hot encoder

In [6]:
onehot_encoder = OneHotEncoder(sparse=False)
label_encoder = LabelEncoder()

integer_encoded = label_encoder.fit_transform(list(nucleic_acids))
onehot_encoder = OneHotEncoder(sparse=False)

integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder.fit(np.asarray(list(nucleic_acids)).reshape(-1, 1))

OneHotEncoder(sparse=False)

In [7]:
# One-hot encode each input string
batch_size = 100
use_channels = False

def encode_and_resize_samples(samples):
  one_hot_samples = np.array([np.array(onehot_encoder.transform(np.asarray(list(sample)).reshape(-1, 1))) for sample in samples])
  if use_channels:
    one_hot_samples2 = np.swapaxes(np.swapaxes(np.expand_dims(one_hot_samples, axis=3), 1, 2), 2, 3)
  else:
    one_hot_samples2 = np.swapaxes(np.expand_dims(one_hot_samples, axis=3), 1, 3)

  return one_hot_samples2

In [8]:
resized_training_samples = encode_and_resize_samples(raw_training_samples)
resized_validation_samples = encode_and_resize_samples(raw_validation_samples)

In [9]:
resized_training_samples[0].shape

(1, 4, 1000)

Reshape and batch one-hot encoded input sequences

In [10]:
def batchify(samples: np.ndarray, batch_size: int) -> np.ndarray:
    for i in list(range(0, len(samples), batch_size)):
      yield samples[i:i+batch_size]

In [11]:
batched_training_samples = list(batchify(resized_training_samples, batch_size))
batched_validation_samples = list(batchify(resized_validation_samples, batch_size))

In [12]:
# Convert one-hot encoded genome samples to PyTorch tensors
training_samples = torch.as_tensor(batched_training_samples).to(device, dtype=torch.float) 
validation_samples = torch.as_tensor(batched_validation_samples).to(device, dtype=torch.float) 

# Set up Pytorch for GPU

In [13]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Train DeepSEA-like model

In [14]:
# Format DeepSEA training data
import h5py
h5f = h5py.File("/gpfs/home/crsmall/CS2952G/ganome/data/deepsea_train/train.mat", "r")

In [15]:
ds_trainx = h5f["trainxdata"][:]
ds_trainy = h5f["traindata"][:]

In [16]:
batch_size = 100

ds_trainx = torch.as_tensor(ds_trainx.reshape((-1, batch_size, 1, 4, 1000))).to(device, dtype=torch.float)
ds_trainy = torch.as_tensor(ds_trainy.reshape((-1, batch_size, 919))).to(device, dtype=torch.float)

In [ ]:
"""
DeepSEA architecture (Zhou & Troyanskaya, 2015).
"""
import numpy as np
import torch
import torch.nn as nn


class DeepSEA(nn.Module):
    def __init__(self):
        super(DeepSEA, self).__init__()
        kernel_size = 8
        num_channels = 1
        n_deepsea_features = 256
        n_output_features = 919

        self.layer1 = nn.Sequential(
          nn.Conv2d(num_channels, 256, (4, kernel_size)),
          nn.LeakyReLU(0.2 ,inplace=True),
          nn.BatchNorm2d(256),
          nn.MaxPool2d((1, 8)),
          nn.Dropout(0.2)
        )
        
        self.layer2 = nn.Sequential(
          nn.Conv2d(256, 512, (1,kernel_size)), # <- This kernel size has to be 1,x for some reason
          nn.LeakyReLU(0.2 ,inplace=True),
          nn.BatchNorm2d(512),
          nn.MaxPool2d((1,8)),
          nn.Dropout(0.2)
        )
        
        self.layer3 = nn.Sequential(
          nn.Conv2d(512, 1028, (1,kernel_size)),
          nn.LeakyReLU(0.2, inplace=True),
          nn.BatchNorm2d(1028),
          nn.Dropout(0.5)
        )
        
        self.linear = nn.Sequential(
          nn.Linear(7196, 919),
          nn.ReLU(True),
          nn.Linear(919, 919),
#           nn.ReLU(True),
#           nn.Linear(1216, n_output_features),
          nn.Sigmoid()
        )

    def forward(self, x):
        f1 = self.layer1(x)
        f2 = self.layer2(f1)
        f3 = self.layer3(f2)
        f3 = f3.view(f3.size(0), -1)
        f4 = self.linear(f3)
        return f4
    
    def print(self, x):
        f1 = self.layer1(x)
        print(f1.size())
        f2 = self.layer2(f1)
        print(f2.size())
        f3 = self.layer3(f2)
        print(f3.size())
        f3 = f3.view(f3.size(0), -1)
        print(f3.size())
        f4 = self.linear(f3)
        print(f4.size())

In [ ]:
deepsea = DeepSEA().to(device)

In [ ]:
deepsea.print(ds_trainx[0])

In [131]:
ds_criterion = nn.BCELoss(reduction="mean")
ds_optimizer = optim.Adam(deepsea.parameters())

In [132]:
num_ds_epochs = 10


for epoch in range(num_ds_epochs):
  for i in range(0, batch_size):
      training_batch = ds_trainx[i]
      training_labels = ds_trainy[i]

      deepsea.zero_grad()
      output = deepsea.forward(training_batch)
        
      training_loss = ds_criterion(output, training_labels)

      training_loss.backward()
      ds_optimizer.step()

      training_loss = training_loss.item()

      print(f"Epoch {epoch}, Batch: {i}: Training Loss: {training_loss}")

Epoch 0, Batch: 0: Training Loss: 0.3015921115875244
Epoch 0, Batch: 1: Training Loss: 0.18967653810977936
Epoch 0, Batch: 2: Training Loss: 0.2592916786670685
Epoch 0, Batch: 3: Training Loss: 0.32434913516044617
Epoch 0, Batch: 4: Training Loss: 0.3612569272518158
Epoch 0, Batch: 5: Training Loss: 0.284496009349823
Epoch 0, Batch: 6: Training Loss: 0.23889963328838348
Epoch 0, Batch: 7: Training Loss: 0.17854973673820496
Epoch 0, Batch: 8: Training Loss: 0.28352242708206177
Epoch 0, Batch: 9: Training Loss: 0.33259519934654236
Epoch 0, Batch: 10: Training Loss: 0.398405522108078
Epoch 0, Batch: 11: Training Loss: 0.28916820883750916
Epoch 0, Batch: 12: Training Loss: 0.46474334597587585
Epoch 0, Batch: 13: Training Loss: 0.25340384244918823
Epoch 0, Batch: 14: Training Loss: 0.1586865484714508
Epoch 0, Batch: 15: Training Loss: 0.36715051531791687
Epoch 0, Batch: 16: Training Loss: 0.31817567348480225
Epoch 0, Batch: 17: Training Loss: 0.24917292594909668
Epoch 0, Batch: 18: Training

Epoch 1, Batch: 51: Training Loss: 0.28364601731300354
Epoch 1, Batch: 52: Training Loss: 0.28666940331459045
Epoch 1, Batch: 53: Training Loss: 0.3105241656303406
Epoch 1, Batch: 54: Training Loss: 0.2626662254333496
Epoch 1, Batch: 55: Training Loss: 0.24946840107440948
Epoch 1, Batch: 56: Training Loss: 0.28590089082717896
Epoch 1, Batch: 57: Training Loss: 0.24682219326496124
Epoch 1, Batch: 58: Training Loss: 0.29141950607299805
Epoch 1, Batch: 59: Training Loss: 0.263651579618454
Epoch 1, Batch: 60: Training Loss: 0.2795623242855072
Epoch 1, Batch: 61: Training Loss: 0.25269731879234314
Epoch 1, Batch: 62: Training Loss: 0.2804722785949707
Epoch 1, Batch: 63: Training Loss: 0.22540879249572754
Epoch 1, Batch: 64: Training Loss: 0.2522360682487488
Epoch 1, Batch: 65: Training Loss: 0.27502039074897766
Epoch 1, Batch: 66: Training Loss: 0.2695813775062561
Epoch 1, Batch: 67: Training Loss: 0.26053300499916077
Epoch 1, Batch: 68: Training Loss: 0.24668380618095398
Epoch 1, Batch: 69

KeyboardInterrupt: 

# Simple NN Generator

In [133]:
num_channels = 1  # Only one channel for genomic data
n_generator_features = 256  # Size of feature maps in generator
n_discriminator_features = 64  # Size of feature maps in discriminator
kernel_size = (4, 10)
input_size = sample_length
ngpu = 1

class Net(nn.Module):
  def __init__(self, ngpu):
    super(Net, self).__init__()
    self.ngpu = ngpu
    self.main = nn.Sequential(
        # First layer
        nn.Conv2d(num_channels, n_generator_features, kernel_size),
        nn.BatchNorm2d(n_generator_features),
        nn.LeakyReLU(0.2 ,inplace=True),
        # Second layer
        nn.Conv2d(n_generator_features, int(n_generator_features/2), (1,10)), # <- This kernel size has to be 1,x for some reason
        nn.BatchNorm2d(int(n_generator_features/2)),
        nn.LeakyReLU(0.2 ,inplace=True),
        nn.ConvTranspose2d(int(n_generator_features/2), n_generator_features, (1, 10)),
        nn.BatchNorm2d(n_generator_features),
        nn.ReLU(True),
        # Output Layer
        nn.ConvTranspose2d(n_generator_features, num_channels, kernel_size),
        nn.Softmax(dim=2)
    )

  def forward(self, input):
    return self.main(input)

In [134]:
# Arbitrarily taken from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [135]:
# Instantiate our model with the detected GPU
net = Net(1).to(device)

# Initialize the weights in our model
# net.apply(weights_init)

# Print the model
print(net)

Net(
  (main): Sequential(
    (0): Conv2d(1, 256, kernel_size=(4, 10), stride=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Conv2d(256, 128, kernel_size=(1, 10), stride=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): ConvTranspose2d(128, 256, kernel_size=(1, 10), stride=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(256, 1, kernel_size=(4, 10), stride=(1, 1))
    (10): Softmax(dim=2)
  )
)


In [ ]:
# # Visualize our model
# from torchviz import make_dot
# y = net(one_hot_samples3)
# make_dot(y)

In [136]:
criterion = nn.BCELoss(reduction='sum')
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters())

In [140]:
HEIGHT = batch_size
WIDTH = 1000
DEPTH = 4

num_epochs = 100

for epoch in range(num_epochs):
  running_loss = 0.0
  for i, training_batch in enumerate(training_samples):
        
    net.zero_grad()
    output = net(training_batch)

    ds_ts = deepsea(training_batch)
    ds_o = deepsea(output)
    
    # Training loss to minimize DeepSEA distances
    criterion = nn.MSELoss(reduction='mean')

    training_loss = criterion(ds_ts, ds_o)

    training_loss.backward()
    optimizer.step()

    training_loss = training_loss.item()

    # Compute validation loss across all validation samples
    validation_total_loss = 0
    for validation_batch in validation_samples:
      validation_output = net(validation_batch)
      validation_total_loss += criterion(validation_output, validation_batch)

    validation_loss = validation_total_loss/num_validation_samples

    print(f"Epoch {epoch}, Batch: {i}: Training Loss: {training_loss}, Validation Loss: {validation_loss}")
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss))
    running_loss = 0.0

Epoch 0, Batch: 0: Training Loss: 8.971385767608808e-08, Validation Loss: 0.003022681688889861
[1,     1] loss: 0.000
Epoch 1, Batch: 0: Training Loss: 6.767019300468746e-08, Validation Loss: 0.0030712741427123547
[2,     1] loss: 0.000
Epoch 2, Batch: 0: Training Loss: 6.954193310093615e-08, Validation Loss: 0.003102491609752178
[3,     1] loss: 0.000


KeyboardInterrupt: 

In [ ]:
y = net(validation_samples[0])

In [ ]:
torch.argmax(validation_samples[0][0], dim=1)

In [ ]:
torch.argmax(y[0], dim=1)

In [ ]:
o = output.detach().cpu().numpy()

In [ ]:
(o == o.max(axis=2)[:,None]).astype(int).reshape(1, 100, 4, 1000).squeeze(0).shape

In [ ]:
deepsea_training_tensor

In [ ]:
deepsea_output_tensor

In [ ]:
len(training_pred_vals)

In [ ]:
!zip -r /content/deepsea_train.zip /content/deepsea_train